In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

c:\Users\Abdulrahman\Desktop\bge_ft_arabic\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "ar_AR"

In [3]:
def translate(ar_text):
  encoded_ar = tokenizer(ar_text, return_tensors="pt")
  generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
  translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  return translated[0]

In [4]:
pairs = pd.read_parquet('data/ar-en.parquet')

In [5]:
pairs['tokens_ar'] = pairs['ar'].progress_apply(lambda x: len(tokenizer(x)['input_ids']))

  0%|          | 0/988979 [00:00<?, ?it/s]

100%|██████████| 988979/988979 [07:04<00:00, 2329.46it/s]


In [6]:
pairs_200 = pairs[pairs['tokens_ar'] < 199]

In [7]:
pairs_200.to_parquet('data/ar-en-200.parquet')

In [8]:
pairs_200['en_translated'] = pairs_200['ar'].progress_apply(lambda x: translate(x))

  0%|          | 2880/988209 [4:27:35<1696:45:27,  6.20s/it]